In [2]:
!pip install -U torchdata
!pip install -U spacy
!python -m spacy download en_core_web_lg
!python -m spacy download de_core_news_lg
!pip install portalocker
!pip install evaluate
!pip install sentence-transformers
!pip install SentencePiece
!pip install bert_score
!pip install --upgrade nltk

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 74.3 MB/s eta 0:00:00
  Attempting uninstall: spacy
    Found existing installation: spacy 3.7.2
    Uninstalling spacy-3.7.2:
      Successfully uninstalled spacy-3.7.2
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 587.7/587.7 MB 2.7 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_lg')
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 567.8/567.8 MB 2.6 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('de_core_news_lg')


In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import json
import torch
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator
from typing import Iterable, List
from torch import Tensor
import torch.nn as nn
from torch.nn import Transformer
import math
from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import DataLoader
from tqdm import tqdm
from timeit import default_timer as timer
from torch.utils.data import Dataset, DataLoader
import scipy
from scipy.stats import pearsonr
from torch.optim.lr_scheduler import LambdaLR
from torch.optim.lr_scheduler import ReduceLROnPlateau
import nltk
import subprocess
import wandb
import torch
from torch.utils.data import DataLoader
import csv
from tqdm.notebook import tqdm
import evaluate
from evaluate import load
import datasets
from datasets import load_dataset
from transformers import T5Tokenizer,T5ForConditionalGeneration
from transformers import pipeline
from tensorboard.backend.event_processing.event_file_loader import EventFileLoader
from tqdm import tqdm
from transformers import pipeline
from nltk.corpus import wordnet

/kaggle/input/modelmd/model2A_epoch_15.pth
/kaggle/input/assignment-3/data/Task1/dev.csv
/kaggle/input/assignment-3/data/Task1/sample_test.csv
/kaggle/input/assignment-3/data/Task1/sample_demo.csv
/kaggle/input/assignment-3/data/Task1/train.csv
/kaggle/input/assignment-3/data/Task2/train_data.json
/kaggle/input/assignment-3/data/Task2/validation_data.json
/kaggle/input/assignment-3/data/Task2/test.json
/kaggle/input/assignment-3/data/A3_task1_data_files/dev.csv
/kaggle/input/assignment-3/data/A3_task1_data_files/sample_test.csv
/kaggle/input/assignment-3/data/A3_task1_data_files/sample_demo.csv
/kaggle/input/assignment-3/data/A3_task1_data_files/train.csv
/kaggle/input/assignment-3/A3_task1_data_files/dev.csv
/kaggle/input/assignment-3/A3_task1_data_files/sample_test.csv
/kaggle/input/assignment-3/A3_task1_data_files/sample_demo.csv
/kaggle/input/assignment-3/A3_task1_data_files/train.csv
/kaggle/input/assignment-3/assignment-3/dev.csv
/kaggle/input/assignment-3/assignment-3/sample_tes

In [ ]:
nltk.download("wordnet",download_dir='/kaggle/working/')
nltk.data.path.append('/kaggle/working/')
try:
    nltk.data.find('/kaggle/working/corpora/wordnet.zip')
except:
    nltk.download('wordnet', download_dir='/kaggle/working/')
    command = "unzip /kaggle/working/corpora/wordnet.zip -d /kaggle/working/corpora"
    subprocess.run(command.split())
    nltk.data.path.append('/kaggle/working/')

In [4]:
wandb_api_key = "e62fc492915628e64fcac9c082089ffed84dc72d"
wandb.login(key=wandb_api_key)
wandb.init(project="nlp3-1a")

wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: karan21258 (karan912). Use `wandb login --relogin` to force relogin
wandb: wandb version 0.16.5 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade
wandb: Tracking run with wandb version 0.16.4
wandb: Run data is saved locally in /kaggle/working/wandb/run-20240330_103745-dim2ti5o
wandb: Run `wandb offline` to turn off syncing.
wandb: Syncing run stellar-resonance-65
wandb: ⭐️ View project at https://wandb.ai/karan912/nlp3-1a
wandb: 🚀 View run at https://wandb.ai/karan912/nlp3-1a/runs/dim2ti5o


In [5]:
token_transform = {}
vocab_transform = {}

In [6]:
SRC_LANGUAGE = 'de'
TGT_LANGUAGE = 'en'

In [ ]:
train_path="/kaggle/input/assignment-3/data/Task2/train_data.json"
val_path="/kaggle/input/assignment-3/data/Task2/validation_data.json"

In [7]:
token_transform[SRC_LANGUAGE] = get_tokenizer('spacy', language='de_core_news_lg')
token_transform[TGT_LANGUAGE] = get_tokenizer('spacy', language='en_core_web_lg')

In [8]:
def yield_tokens(data_iter: Iterable, language: str) -> List[str]:
    language_index = {SRC_LANGUAGE: 0, TGT_LANGUAGE: 1}

    for data_sample in data_iter:
        yield token_transform[language](data_sample[language_index[language]])

In [9]:
def read_json(file_path):
    with open(file_path, 'r', encoding='utf-8') as f:
        data = json.load(f)
    print(len(data["translation"]))
    return data["translation"]

In [10]:
class CustomDataset(Dataset):
    def __init__(self, json_data):
        self.data = []
        for item in json_data:
            # Assuming each item in JSON is a dictionary with keys 'input' and 'target'
            input_data = item[SRC_LANGUAGE]
            target_data = item[TGT_LANGUAGE]
            self.data.append((input_data, target_data))

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return self.data[idx]

In [11]:
UNK_IDX, PAD_IDX, BOS_IDX, EOS_IDX = 0, 1, 2, 3
# Make sure the tokens are in order of their indices to properly insert them in vocab
special_symbols = ['<unk>', '<pad>', '<bos>', '<eos>']

def getVocab_train(file_path):
    for ln in [SRC_LANGUAGE, TGT_LANGUAGE]:
        # Create train_iter from JSON file
        iterator = CustomDataset(read_json(file_path))
        vocab_transform[ln] = build_vocab_from_iterator(yield_tokens(iterator, ln),
                                                        min_freq=1,
                                                        specials=special_symbols,
                                                        special_first=True)
    return iterator

train_iter = getVocab_train(train_path)

for ln in [SRC_LANGUAGE, TGT_LANGUAGE]:
    vocab_transform[ln].set_default_index(UNK_IDX)
        


50000
50000


In [12]:
val_iter = CustomDataset(read_json(val_path))
print(val_iter)

2169


In [14]:
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# helper Module that adds positional encoding to the token embedding to introduce a notion of word order.
class PositionalEncoding(nn.Module):
    def __init__(self,
                 emb_size: int,
                 dropout: float,
                 maxlen: int = 5000):
        super(PositionalEncoding, self).__init__()
        den = torch.exp(- torch.arange(0, emb_size, 2)* math.log(10000) / emb_size)
        pos = torch.arange(0, maxlen).reshape(maxlen, 1)
        pos_embedding = torch.zeros((maxlen, emb_size))
        pos_embedding[:, 0::2] = torch.sin(pos * den)
        pos_embedding[:, 1::2] = torch.cos(pos * den)
        pos_embedding = pos_embedding.unsqueeze(-2)

        self.dropout = nn.Dropout(dropout)
        self.register_buffer('pos_embedding', pos_embedding)

    def forward(self, token_embedding: Tensor):
        return self.dropout(token_embedding + self.pos_embedding[:token_embedding.size(0), :])

# helper Module to convert tensor of input indices into corresponding tensor of token embeddings
class TokenEmbedding(nn.Module):
    def __init__(self, vocab_size: int, emb_size):
        super(TokenEmbedding, self).__init__()
        self.embedding = nn.Embedding(vocab_size, emb_size)
        self.emb_size = emb_size

    def forward(self, tokens: Tensor):
        return self.embedding(tokens.long()) * math.sqrt(self.emb_size)

# Seq2Seq Network
class Seq2SeqTransformer(nn.Module):
    def __init__(self,
                 num_encoder_layers: int,
                 num_decoder_layers: int,
                 emb_size: int,
                 nhead: int,
                 src_vocab_size: int,
                 tgt_vocab_size: int,
                 dim_feedforward: int = 512,
                 dropout: float = 0.1):
        super(Seq2SeqTransformer, self).__init__()
        self.transformer = Transformer(d_model=emb_size,
                                       nhead=nhead,
                                       num_encoder_layers=num_encoder_layers,
                                       num_decoder_layers=num_decoder_layers,
                                       dim_feedforward=dim_feedforward,
                                       dropout=dropout)
        self.generator = nn.Linear(emb_size, tgt_vocab_size)
        self.src_tok_emb = TokenEmbedding(src_vocab_size, emb_size)
        self.tgt_tok_emb = TokenEmbedding(tgt_vocab_size, emb_size)
        self.positional_encoding = PositionalEncoding(
            emb_size, dropout=dropout)

    def forward(self,
                src: Tensor,
                trg: Tensor,
                src_mask: Tensor,
                tgt_mask: Tensor,
                src_padding_mask: Tensor,
                tgt_padding_mask: Tensor,
                memory_key_padding_mask: Tensor):
        src_emb = self.positional_encoding(self.src_tok_emb(src))
        tgt_emb = self.positional_encoding(self.tgt_tok_emb(trg))
        outs = self.transformer(src_emb, tgt_emb, src_mask, tgt_mask, None,
                                src_padding_mask, tgt_padding_mask, memory_key_padding_mask)
        return self.generator(outs)

    def encode(self, src: Tensor, src_mask: Tensor):
        return self.transformer.encoder(self.positional_encoding(
                            self.src_tok_emb(src)), src_mask)

    def decode(self, tgt: Tensor, memory: Tensor, tgt_mask: Tensor):
        return self.transformer.decoder(self.positional_encoding(
                          self.tgt_tok_emb(tgt)), memory,
                          tgt_mask)

In [15]:
def generate_square_subsequent_mask(sz):
    mask = (torch.triu(torch.ones((sz, sz), device=DEVICE)) == 1).transpose(0, 1)
    mask = mask.float().masked_fill(mask == 0, float('-inf')).masked_fill(mask == 1, float(0.0))
    return mask


def create_mask(src, tgt):
    src_seq_len = src.shape[0]
    tgt_seq_len = tgt.shape[0]

    tgt_mask = generate_square_subsequent_mask(tgt_seq_len)
    src_mask = torch.zeros((src_seq_len, src_seq_len),device=DEVICE).type(torch.bool)

    src_padding_mask = (src == PAD_IDX).transpose(0, 1)
    tgt_padding_mask = (tgt == PAD_IDX).transpose(0, 1)
    return src_mask, tgt_mask, src_padding_mask, tgt_padding_mask

In [16]:
torch.cuda.empty_cache()
torch.manual_seed(0)

SRC_VOCAB_SIZE = len(vocab_transform[SRC_LANGUAGE])
TGT_VOCAB_SIZE = len(vocab_transform[TGT_LANGUAGE])
print(SRC_VOCAB_SIZE)
print(TGT_VOCAB_SIZE)
EMB_SIZE = 512
NHEAD = 8
FFN_HID_DIM = 16
BATCH_SIZE = 40
NUM_ENCODER_LAYERS = 4
NUM_DECODER_LAYERS = 4
DROPOUT = 0.03

transformer = Seq2SeqTransformer(NUM_ENCODER_LAYERS, NUM_DECODER_LAYERS, EMB_SIZE,
                                 NHEAD, SRC_VOCAB_SIZE, TGT_VOCAB_SIZE, FFN_HID_DIM, DROPOUT)

for p in transformer.parameters():
    if p.dim() > 1:
        nn.init.xavier_uniform_(p)

transformer = transformer.to(DEVICE)

loss_fn = torch.nn.CrossEntropyLoss(ignore_index=PAD_IDX)


52604
23774


/opt/conda/lib/python3.10/site-packages/torch/nn/modules/transformer.py:282: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")


In [17]:
# helper function to club together sequential operations
def sequential_transforms(*transforms):
    def func(txt_input):
        for transform in transforms:
            txt_input = transform(txt_input)
        return txt_input
    return func

# function to add BOS/EOS and create tensor for input sequence indices
def tensor_transform(token_ids: List[int]):
    return torch.cat((torch.tensor([BOS_IDX]),
                      torch.tensor(token_ids),
                      torch.tensor([EOS_IDX])))

# ``src`` and ``tgt`` language text transforms to convert raw strings into tensors indices
text_transform = {}
for ln in [SRC_LANGUAGE, TGT_LANGUAGE]:
    text_transform[ln] = sequential_transforms(token_transform[ln], #Tokenization
                                               vocab_transform[ln], #Numericalization
                                               tensor_transform) # Add BOS/EOS and create tensor


# function to collate data samples into batch tensors
def collate_fn(batch):
    src_batch, tgt_batch = [], []
    for src_sample, tgt_sample in batch:
        src_batch.append(text_transform[SRC_LANGUAGE](src_sample.rstrip("\n")))
        tgt_batch.append(text_transform[TGT_LANGUAGE](tgt_sample.rstrip("\n")))

    src_batch = pad_sequence(src_batch, padding_value=PAD_IDX)
    tgt_batch = pad_sequence(tgt_batch, padding_value=PAD_IDX)
    return src_batch, tgt_batch

In [18]:

    
def train_epoch(model, optimizer,epoch):
    model.train()
    losses = 0
    train_dataloader = DataLoader(train_iter, batch_size=BATCH_SIZE, collate_fn=collate_fn)
    train_dataloader = tqdm(train_dataloader, desc=f"Epoch:{epoch}", leave=False)
    predicted_outputs = []

    for batch in train_dataloader:
        src = batch[0]
        tgt = batch[1]
        src = src.to(DEVICE)
        tgt = tgt.to(DEVICE)

        tgt_input = tgt[:-1, :]

        src_mask, tgt_mask, src_padding_mask, tgt_padding_mask = create_mask(src, tgt_input)

        logits = model(src, tgt_input, src_mask, tgt_mask,src_padding_mask, tgt_padding_mask, src_padding_mask)

        optimizer.zero_grad()

        tgt_out = tgt[1:, :]
        loss = loss_fn(logits.reshape(-1, logits.shape[-1]), tgt_out.reshape(-1))
        loss.backward()

        optimizer.step()
        losses += loss.item()
#         scheduler.step()
        
    x = losses / len(list(train_dataloader))
    tqdm.write(f"Epoch:{epoch}, Avg Train Loss: {x}")
    
    return x



def evaluate(model):
    model.eval()
    losses = 0
    val_dataloader = DataLoader(val_iter, batch_size=BATCH_SIZE, collate_fn=collate_fn)

    for src, tgt in val_dataloader:
        src = src.to(DEVICE)
        tgt = tgt.to(DEVICE)

        tgt_input = tgt[:-1, :]

        src_mask, tgt_mask, src_padding_mask, tgt_padding_mask = create_mask(src, tgt_input)

        logits = model(src, tgt_input, src_mask, tgt_mask,src_padding_mask, tgt_padding_mask, src_padding_mask)

        tgt_out = tgt[1:, :]
        loss = loss_fn(logits.reshape(-1, logits.shape[-1]), tgt_out.reshape(-1))
        losses += loss.item()

    return losses / len(list(val_dataloader))

In [19]:
NUM_EPOCHS = 20

for epoch in range(1, NUM_EPOCHS+1):
    start_time = timer()
    train_loss = train_epoch(transformer, optimizer,epoch)
    end_time = timer()
    val_loss = evaluate(transformer)
    torch.save(transformer.state_dict(), f"model2A_epoch_{epoch}.pth")
    print((f"Epoch: {epoch}, Train loss: {train_loss:.3f}, Val loss: {val_loss:.3f}, "f"Epoch time = {(end_time - start_time):.3f}s",f"current_lr = {optimizer.param_groups[0]['lr']}"))
#     scheduler.step(val_loss)
    wandb.log({"Epoch": epoch, "Train loss": f"{train_loss:.3f}", "Val loss": f"{val_loss:.3f}", "Epoch time":f"{(end_time - start_time):.3}","lr":optimizer.param_groups[0]['lr']})
# function to generate output sequence using greedy algorithm
def greedy_decode(model, src, src_mask, max_len, start_symbol):
    src = src.to(DEVICE)
    src_mask = src_mask.to(DEVICE)

    memory = model.encode(src, src_mask)
    ys = torch.ones(1, 1).fill_(start_symbol).type(torch.long).to(DEVICE)
    for i in range(max_len-1):
        memory = memory.to(DEVICE)
        tgt_mask = (generate_square_subsequent_mask(ys.size(0))
                    .type(torch.bool)).to(DEVICE)
        out = model.decode(ys, memory, tgt_mask)
        out = out.transpose(0, 1)
        prob = model.generator(out[:, -1])
        _, next_word = torch.max(prob, dim=1)
        next_word = next_word.item()

        ys = torch.cat([ys,
                        torch.ones(1, 1).type_as(src.data).fill_(next_word)], dim=0)
        if next_word == EOS_IDX:
            break
    return ys


# actual function to translate input sentence into target language
def translate(model: torch.nn.Module, src_sentence: str):
    model.eval()
    src = text_transform[SRC_LANGUAGE](src_sentence).view(-1, 1)
    num_tokens = src.shape[0]
    src_mask = (torch.zeros(num_tokens, num_tokens)).type(torch.bool)
    tgt_tokens = greedy_decode(
        model,  src, src_mask, max_len=num_tokens + 5, start_symbol=BOS_IDX).flatten()
    return " ".join(vocab_transform[TGT_LANGUAGE].lookup_tokens(list(tgt_tokens.cpu().numpy()))).replace("<bos>", "").replace("<eos>", "")

Epoch:1:   0%|          | 0/1250 [00:00<?, ?it/s]/opt/conda/lib/python3.10/site-packages/torch/nn/functional.py:5076: UserWarning: Support for mismatched key_padding_mask and attn_mask is deprecated. Use same type for both instead.
  warnings.warn(


Epoch:1, Avg Train Loss: 6.024660729980469
('Epoch: 1, Train loss: 6.025, Val loss: 6.900, Epoch time = 179.271s', 'current_lr = 4e-05')


Epoch:2, Avg Train Loss: 4.9412616275787355
('Epoch: 2, Train loss: 4.941, Val loss: 6.545, Epoch time = 178.259s', 'current_lr = 4e-05')


Epoch:3, Avg Train Loss: 4.55308837184906
('Epoch: 3, Train loss: 4.553, Val loss: 6.333, Epoch time = 178.358s', 'current_lr = 4e-05')


Epoch:4, Avg Train Loss: 4.244938721466064
('Epoch: 4, Train loss: 4.245, Val loss: 6.127, Epoch time = 178.027s', 'current_lr = 4e-05')


...

Epoch:16, Avg Train Loss: 2.273909913444519
('Epoch: 16, Train loss: 2.274, Val loss: 6.251, Epoch time = 178.591s', 'current_lr = 4e-05')


Epoch:17, Avg Train Loss: 2.1740304832458497
('Epoch: 17, Train loss: 2.174, Val loss: 6.366, Epoch time = 178.601s', 'current_lr = 4e-05')


Epoch:18, Avg Train Loss: 2.0770499737739563
('Epoch: 18, Train loss: 2.077, Val loss: 6.418, Epoch time = 178.754s', 'current_lr = 4e-05')


Epoch:19, Avg Train Loss: 1.98667237701416
('Epoch: 19, Train loss: 1.987, Val loss: 6.428, Epoch time = 178.638s', 'current_lr = 4e-05')


Epoch:20, Avg Train Loss: 1.900699625492096
('Epoch: 20, Train loss: 1.901, Val loss: 6.441, Epoch time = 178.564s', 'current_lr = 4e-05')


In [20]:
print(translate(transformer, "Die Premierminister Indiens und Japans trafen sich in Tokio."))

 The Prime Minister for Minister to say that I am sure the Prime Minister is concerned


In [22]:
def generate_predictions(model, data_iter, DEVICE, collate_fn):
#     model.eval()
    predicted_outputs = []
    de_input = []
    data_dataloader = DataLoader(data_iter, batch_size=1, collate_fn=collate_fn)
#     print(data_dataloader)
    x = tqdm(data_iter,desc= "predicting", total = len(data_iter))
    i=1
    for batch in x:
        i+=1
        predictions = translate(model, batch[0])
        predicted_outputs.append((batch[0],predictions))
        de_input.append(batch[0])
    return predicted_outputs, de_input



def save_predicted_outputs(outputs, filename):
    with open(filename, 'w', newline='') as csvfile:

        # Define the fieldnames for the CSV
        fieldnames = ['de', 'en']
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

        # Write the header row
        writer.writeheader()
        for item in outputs:
            writer.writerow({'de': item[0], 'en': item[1]})

# Assuming you have your data iterator ready, e.g., `test_iter`
predicted_outputs, de_test_data = generate_predictions(transformer, val_iter, DEVICE, collate_fn)

# Save predicted outputs to a CSV file
save_predicted_outputs(predicted_outputs, "predicted_outputs.csv")

predicting:   0%|          | 0/2169 [00:00<?, ?it/s]

In [23]:
!pip install evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 5.3 MB/s eta 0:00:00


In [24]:
! pip install bert_score

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 3.9 MB/s eta 0:00:00


In [25]:
! pip install --upgrade nltk

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 54.2 MB/s eta 0:00:00
  Attempting uninstall: nltk
    Found existing installation: nltk 3.2.4
    Uninstalling nltk-3.2.4:
      Successfully uninstalled nltk-3.2.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
preprocessing 0.1.13 requires nltk==3.2.4, but you have nltk 3.8.1 which is incompatible.


In [26]:
! pip install SentencePiece
! pip install sentence-transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 163.3/163.3 kB 10.8 MB/s eta 0:00:00


In [27]:
! pip install evaluate
! pip install sentence-transformers
! pip install SentencePiece
! pip install bert_score
! pip install --upgrade nltk

In [28]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import evaluate
from evaluate import load
import datasets
from datasets import load_dataset

from transformers import T5Tokenizer,T5ForConditionalGeneration
import json

import torch

from transformers import pipeline

from tensorboard.backend.event_processing.event_file_loader import EventFileLoader
import matplotlib.pyplot as plt

import csv
from tqdm import tqdm
from transformers import pipeline

2024-03-30 11:46:40.085777: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-30 11:46:40.085885: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-30 11:46:40.209832: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [29]:
predicted_outputs2 = [i[1] for i in predicted_outputs]

In [30]:
en_test_data = [i[1] for i in val_iter]


In [31]:
from evaluate import load
import evaluate
bertscore = load("bertscore")
meteor = evaluate.load("meteor")
bleu = evaluate.load("bleu")

[nltk_data] Downloading package wordnet to /usr/share/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /usr/share/nltk_data...


In [32]:
def calculate_bleu(bp,precisions):
    weights_1 = np.array([1,0,0,0])
    weights_2 = np.array([0.5,0.5,0,0])
    weights_3 = np.array([0.33,0.33,0.33,0])
    weights_4 = np.array([0.25,0.25,0.25,0.25])
    logp = np.log(precisions)
        
    res1 = bp*np.exp(np.dot(logp,weights_1))
    res2 = bp*np.exp(np.dot(logp,weights_2))
    res3 = bp*np.exp(np.dot(logp,weights_3))
    res4 = bp*np.exp(np.dot(logp,weights_4))
    
    return [res1,res2,res3,res4]

In [33]:
bleu_score = bleu.compute(predictions=predicted_outputs2, references=en_test_data)
results = calculate_bleu(bleu_score["brevity_penalty"],bleu_score["precisions"])

print("Bleu Score : ")
print(bleu_score)
print("Bleu-1 : ",results[0])
print("Bleu-2 : ",results[1])
print("Bleu-3 : ",results[2])
print("Bleu-4 : ",results[3])

Bleu Score : 
{'bleu': 0.040953962062909875, 'precisions': [0.2925642807505212, 0.06480313108197704, 0.020049788064320798, 0.007400424228140467], 'brevity_penalty': 1.0, 'length_ratio': 1.0534633852248994, 'translation_length': 48926, 'reference_length': 46443}
Bleu-1 :  0.2925642807505212
Bleu-2 :  0.13769198028708993
Bleu-3 :  0.07436625917381377
Bleu-4 :  0.04095396206290987


In [38]:
results = meteor.compute(predictions=predicted_outputs2, references=en_test_data)
print("Meteor Score : ",results)

Meteor Score :  {'meteor': 0.22598777555662705}


In [39]:
results = bertscore.compute(predictions=predicted_outputs2, references=en_test_data, lang="en")
print("Bert Score : ")
print("F1 = ",np.average(results["f1"]))
print("Precision = ",np.average(results["precision"]))
print("Recall = ",np.average(results["recall"]))

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/482 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.42G [00:00<?, ?B/s]

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Bert Score : 
F1 =  0.8398771589942218
Precision =  0.8331448033747446
Recall =  0.8469450646086517


In [40]:
wandb.finish()

wandb:                                                                                
wandb: 
wandb: Run history:
wandb: Epoch ▁▁▂▂▂▃▃▄▄▄▅▅▅▆▆▇▇▇██
wandb:    lr ▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
wandb: 
wandb: Run summary:
wandb:      Epoch 20
wandb: Epoch time 1.79e+02
wandb: Train loss 1.901
wandb:   Val loss 6.441
wandb:         lr 4e-05
wandb: 
wandb: 🚀 View run stellar-resonance-65 at: https://wandb.ai/karan912/nlp3-1a/runs/dim2ti5o
wandb: Synced 6 W&B file(s), 0 media file(s), 0 artifact file(s) and 0 other file(s)
wandb: Find logs at: ./wandb/run-20240330_103745-dim2ti5o/logs


In [41]:
torch.cuda.empty_cache()